# TS - Final Project

In [112]:
import pandas as pd
import numpy as np
import math
import random

from copy import deepcopy
from scipy import interpolate
import numpy as np
#from dtaidistance import dtw
import matplotlib.pyplot as plt
from _plotly_future_ import v4_subplots
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Data Processing

In [2]:
# import data
df = pd.read_csv('billboardHot100_1999-2019.csv')
df = df.rename(columns={"Weekly.rank": "WeeklyRank"})

In [3]:
# parse out genre column for each song
genres = df.Genre.str.split(',', expand=True)
genres

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Country,Atlanta,Alternative Country,Hip-Hop,Trap,Memes,Remix,Country Rap,Rap,None,None,None,None,None,None,None
1,Pop,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Hip-Hop,Dark Pop,House,Trap,Memes,Alternative Dance,Alternative,Alternative Pop,Dance-Pop,Dance,Electro-Pop,Electro,Pop,None,None,None
3,Synth-Pop,Pop,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Canada,UK,Dance,Dance-Pop,Pop,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97220,Jamaica,Pop,None,None,None,None,None,None,None,None,None,None,None,None,None,None
97221,Hockey,Gaming,Soundtrack,Rock,None,None,None,None,None,None,None,None,None,None,None,None
97222,Country,Pop,None,None,None,None,None,None,None,None,None,None,None,None,None,None
97223,Rap,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [4]:
# set week & rank as indexes
cols = ['Week','WeeklyRank']
tuple_id = list(zip(df.Week, df.WeeklyRank))
idx = pd.MultiIndex.from_tuples(tuple_id, names=cols)
# create dataframe with week, rank, and all parsed out genres
ts_genre = pd.concat([df[cols], genres], axis=1).set_index(idx)
ts_genre = ts_genre.drop(cols, axis=1)
ts_genre

0          1                    2          3   \
Week       WeeklyRank                                                         
2019-07-06 1             Country    Atlanta  Alternative Country    Hip-Hop   
           2                 Pop       None                 None       None   
           3             Hip-Hop   Dark Pop                House       Trap   
           4           Synth-Pop        Pop                 None       None   
           5              Canada         UK                Dance  Dance-Pop   
...                          ...        ...                  ...        ...   
1999-07-12 95            Jamaica        Pop                 None       None   
           96             Hockey     Gaming           Soundtrack       Rock   
           97            Country        Pop                 None       None   
           98                Rap       None                 None       None   
           100               Ska  Punk Rock                 Rock       None   

                          4                  5            6                7   \
Week       WeeklyRank                                                           
2019-07-06 1            Trap              Memes        Remix      Country Rap   
           2            None               None         None             None   
           3           Memes  Alternative Dance  Alternative  Alternative Pop   
           4            None               None         None             None   
           5             Pop               None         None             None   
...                      ...                ...          ...              ...   
1999-07-12 95           None               None         None             None   
           96           None               None         None             None   
           97           None               None         None             None   
           98           None               None         None             None   
           100          None               None         None             None   

                              8      9            10       11    12    13  \
Week       WeeklyRank                                                       
2019-07-06 1                 Rap   None         None     None  None  None   
           2                None   None         None     None  None  None   
           3           Dance-Pop  Dance  Electro-Pop  Electro   Pop  None   
           4                None   None         None     None  None  None   
           5                None   None         None     None  None  None   
...                          ...    ...          ...      ...   ...   ...   
1999-07-12 95               None   None         None     None  None  None   
           96               None   None         None     None  None  None   
           97               None   None         None     None  None  None   
           98               None   None         None     None  None  None   
           100              None   None         None     None  None  None   

                         14    15  
Week       WeeklyRank              
2019-07-06 1           None  None  
           2           None  None  
           3           None  None  
           4           None  None  
           5           None  None  
...                     ...   ...  
1999-07-12 95          None  None  
           96          None  None  
           97          None  None  
           98          None  None  
           100         None  None  

[97225 rows x 16 columns]

In [5]:
# stack genres
final_ts_genre = ts_genre.stack().reset_index().drop(['level_2'], axis=1)
print(final_ts_genre.shape)
final_ts_genre.head(10)

(277464, 3)


,Week,WeeklyRank,0
0,2019-07-06,1,Country
1,2019-07-06,1,Atlanta
2,2019-07-06,1,Alternative Country
3,2019-07-06,1,Hip-Hop
4,2019-07-06,1,Trap
5,2019-07-06,1,Memes
6,2019-07-06,1,Remix
7,2019-07-06,1,Country Rap
8,2019-07-06,1,Rap
9,2019-07-06,2,Pop


### Check

In [6]:
final_ts_genre.tail(3)

,Week,WeeklyRank,0
277461,1999-07-12,100,Ska
277462,1999-07-12,100,Punk Rock
277463,1999-07-12,100,Rock


In [7]:
df.tail(1)

,Unnamed: 0,Artists,Name,WeeklyRank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
97224,97225,The Offspring,Why Don't You Get A Job?,100,74.0,15.0,1999-07-12,NaN,"Ska,Punk Rock,Rock",Dexter holland,Why Don't You Get a Job? \nMy friend's got a g...,NaN


### Rank by Genre by day

In [14]:
final_ts_genre_week = final_ts_genre.groupby(['Week',0], as_index=False)['WeeklyRank'].mean()
final_ts_genre_week = final_ts_genre_week.rename(columns={0: "Genre"})
final_ts_genre_week

,Week,Genre,WeeklyRank
0,1999-07-12,Adult Alternative,21.000000
1,1999-07-12,Adult Contemporary,15.000000
2,1999-07-12,Alternative Rock,32.600000
3,1999-07-12,Ballad,35.000000
4,1999-07-12,Blues Rock,45.000000
...,...,...,...
62548,2019-07-06,Trap,38.166667
62549,2019-07-06,Trip-Hop,49.000000
62550,2019-07-06,Tropical House,100.000000
62551,2019-07-06,UK,33.666667


In [16]:
## Check
final_ts_genre_week[final_ts_genre_week['Week'] == '1999-07-12'].sort_values('WeeklyRank').head(10)

,Week,Genre,WeeklyRank
9,1999-07-12,Dance-Pop,4.0
19,1999-07-12,Latin Pop,5.0
16,1999-07-12,Girl Group,9.0
20,1999-07-12,Memes,10.0
35,1999-07-12,Ska Punk,10.0
26,1999-07-12,Power Pop,10.0
1,1999-07-12,Adult Contemporary,15.0
11,1999-07-12,Disney,21.0
0,1999-07-12,Adult Alternative,21.0
40,1999-07-12,Teen Pop,22.0


In [20]:
# check - value above should be average of values below
final_ts_genre[final_ts_genre['Week'] == '1999-07-12'].sort_values(0).head(3)

,Week,WeeklyRank,0
277302,1999-07-12,15,Adult Alternative
277367,1999-07-12,45,Adult Alternative
277272,1999-07-12,3,Adult Alternative


## Export Data

In [17]:
final_ts_genre_week.to_csv(r'C:\Users\willm\OneDrive\Documents\MSCA\31006 - Time Series\Final Project\FinalData.csv', index = False)

# 2. Dynamic Time Warping - Genres

In [129]:
# genre dataframes
Rock = final_ts_genre_week[final_ts_genre_week['Genre'] == 'Rock']['WeeklyRank']
Pop = final_ts_genre_week[final_ts_genre_week['Genre'] == 'Pop']['WeeklyRank']
Country = final_ts_genre_week[final_ts_genre_week['Genre'] == 'Country']['WeeklyRank']
Rap = final_ts_genre_week[final_ts_genre_week['Genre'] == 'Rap']['WeeklyRank']
RandB = final_ts_genre_week[final_ts_genre_week['Genre'] == 'R&;B']['WeeklyRank']

In [261]:
final_ts_genre_week['Genre'] = final_ts_genre_week['Genre'].replace(' ','', regex = True)
final_ts_genre_week['Genre'] = final_ts_genre_week['Genre'].replace('-', '', regex = True)
final_ts_genre_week['Genre'] = final_ts_genre_week['Genre'].replace('&;', 'and', regex = True)
final_ts_genre_week['Genre'] = final_ts_genre_week['Genre'].replace('\xc3\xb1ol', '', regex = True)
final_ts_genre_week['Genre']

0         AdultAlternative
1        AdultContemporary
2          AlternativeRock
3                   Ballad
4                BluesRock
               ...        
62548                 Trap
62549              TripHop
62550        TropicalHouse
62551                   UK
62552            WestCoast
Name: Genre, Length: 62553, dtype: object

In [262]:
final_ts_genre_week['Genre'].value_counts().head(50).index

Index(['Rap', 'Country', 'Pop', 'Rock', 'RandB', 'AlternativeRock',
       'AdultAlternative', 'PopRock', 'AdultContemporary', 'Soul', 'Ballad',
       'Producer', 'HipHop', 'Alternative', 'Soundtrack', 'Electronic',
       'DancePop', 'SoulPop', 'Memes', 'Dance', 'UK', 'Canada', 'WestCoast',
       'ElectroPop', 'IndieRock', 'EastCoast', 'Remix', 'Screen', 'SynthPop',
       'Piano', 'AlternativePop', 'PopPunk', 'NeoSoul', 'Trap', 'Dancehall',
       'PopCountry', 'PowerPop', 'PostGrunge', 'Folk', 'ElectronicRock',
       'SingerSongwriter', 'DMV', 'IndiePop', 'LatinPop', 'Reggae',
       'EnEspa\xc3\xb1ol', 'Atlanta', 'PunkRock', 'DirtySouth', 'Acoustic'],
      dtype='object')

In [280]:
top_genres_str = final_ts_genre_week['Genre'].value_counts().head(50).index
top_genres = [Rap, Country, Pop, Rock, RandB, AlternativeRock,AdultAlternative, PopRock, AdultContemporary, Soul, Ballad,
       Producer, HipHop, Alternative, Soundtrack, Electronic,DancePop, SoulPop, Memes, Dance, UK, Canada, WestCoast,
       ElectroPop, IndieRock, EastCoast, Remix, Screen, SynthPop,Piano, AlternativePop, PopPunk, NeoSoul, Trap, Dancehall,
       PopCountry, PowerPop, PostGrunge, Folk, ElectronicRock,SingerSongwriter, DMV, IndiePop, LatinPop, Reggae, Atlanta, PunkRock, DirtySouth, Acoustic]

In [264]:
for genre in top_genres_clean:
    globals()[genre] = final_ts_genre_week[final_ts_genre_week['Genre'] == genre]['WeeklyRank']

In [94]:
genres = [Rock,Pop,Country,RandB,Rap]
genres_str = ['Rock','Pop','Country','RandB','Rap']

### Functions

In [99]:
def DTWDistance(s1, s2):
    DTW={}

    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            dist= (s1.iloc[i]-s2.iloc[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return math.sqrt(DTW[len(s1)-1, len(s2)-1])

In [84]:
DTWDistance(Rock, Pop)

171.72643737187596

In [85]:
DTWDistance(Rap, Pop)

110.0702437061613

In [87]:
DTWDistance(Country, Rap)

193.80245000220341

In [64]:
dtw_genre_matrix = np.zeros((len(genres), len(genres)))
dtw_genre_matrix

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [89]:
for idx1,genre1 in enumerate(genres):
    for idx2,genre2 in enumerate(genres):
        distance = DTWDistance(genre1,genre2)
        dtw_genre_matrix[idx1,idx2] = distance

In [96]:
dtw_genre_df = pd.DataFrame(dtw_genre_matrix)
dtw_genre_df.columns = genres_str
dtw_genre_df.index = genres_str
dtw_genre_df

,Rock,Pop,Country,RandB,Rap
Rock,0.000000,171.726437,257.747426,167.619333,186.001844
Pop,171.726437,0.000000,391.124372,117.302986,110.070244
Country,257.747426,391.124372,0.000000,268.265438,193.802450
RandB,167.619333,117.302986,268.265438,0.000000,127.873213
Rap,186.001844,110.070244,193.802450,127.873213,0.000000


### Add window constraint to DTW
#### Only mappings within window are considered which speeds up inner loop

In [152]:
def DTWDistance_w(s1, s2,w):
    DTW={}

    w = max(w, abs(len(s1)-len(s2)))

    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return math.sqrt(DTW[len(s1)-1, len(s2)-1])

In [103]:
dtw_genre_matrix2 = np.zeros((len(genres), len(genres)))
for idx1,genre1 in enumerate(genres):
    for idx2,genre2 in enumerate(genres):
        distance = DTWDistance_w(genre1,genre2, 104)
        dtw_genre_matrix2[idx1,idx2] = distance

In [104]:
dtw_genre_df2 = pd.DataFrame(dtw_genre_matrix2)
dtw_genre_df2.columns = genres_str
dtw_genre_df2.index = genres_str
dtw_genre_df2

,Rock,Pop,Country,RandB,Rap
Rock,0.000000,272.120086,274.593471,213.317276,224.047610
Pop,271.710335,0.000000,468.997411,133.543106,127.923571
Country,274.192626,469.041759,0.000000,398.309428,314.090569
RandB,213.237669,133.877281,398.164220,0.000000,132.079055
Rap,224.047610,128.174943,313.533890,132.079055,0.000000


# 3. Clustering
### Using DTW as measure of similarity

#### DTW function that works for np arrays, uses windows to speed up computation

In [156]:
def DTWDistance_w(s1, s2,w):
    DTW={}

    w = max(w, abs(s1.itemsize-s2.itemsize))

    for i in range(-1,s1.itemsize):
        for j in range(-1,s2.itemsize):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(s1.itemsize):
        for j in range(max(0, i-w), min(s2.itemsize, i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return math.sqrt(DTW[s1.itemsize-1, s2.itemsize-1])

#### create dictionary of genres and weekly ranks

In [285]:
genreSet = {}
for idx, genre in enumerate(top_genres):
    genreSet[(str(idx),)] = [np.array(genre)]
genreSet

{('0',): [array([47.58823529, 47.64705882, 50.38888889, ..., 50.        ,
         49.23684211, 49.73170732])],
 ('1',): [array([59.52380952, 60.57142857, 59.9       , ..., 53.82608696,
         54.95238095, 54.9047619 ])],
 ('2',): [array([47.81081081, 49.60526316, 49.10526316, ..., 46.39583333,
         45.6875    , 45.65957447])],
 ('3',): [array([47.53846154, 43.08333333, 45.61538462, ..., 21.5       ,
         25.5       , 32.5       ])],
 ('4',): [array([36.85      , 37.76190476, 34.95      , ..., 46.45      ,
         47.55      , 51.55      ])],
 ('5',): [array([32.6       , 33.        , 40.33333333, ..., 27.33333333,
         29.        , 35.66666667])],
 ('6',): [array([21.        , 21.        , 19.33333333, ..., 19.5       ,
         21.5       , 28.5       ])],
 ('7',): [array([27.  , 24.75, 36.2 , ..., 26.8 , 28.4 , 31.8 ])],
 ('8',): [array([15., 14., 17., ..., 18., 20., 25.])],
 ('9',): [array([45.        , 47.5       , 43.375     , 42.25      , 41.        ,
         38.

#### calculate distances and cluster genres that are below threshold distance

In [286]:
genre_ts = deepcopy(genreSet)
distanceMatrixDict = {}
THRESHOLD = 15

iteration = 1
while True:
    ###### CALCULATE DISTANCES #########
    distanceMatrix_2 = np.empty((len(genre_ts), len(genre_ts),))
    distanceMatrix_2[:] = np.nan
    
    for index1, (filter1, trajectory1) in enumerate(genre_ts.items()):
        tempArray = []
        
        for index2, (filter2, trajectory2) in enumerate(genre_ts.items()):
            
            if index1 > index2:
                continue
            
            elif index1 == index2:
                continue
            
            else:
                unionFilter = filter1 + filter2
                sorted(unionFilter)
                
                if unionFilter in distanceMatrixDict.keys():
                    distanceMatrix_2[index1][index2] = distanceMatrixDict.get(unionFilter)
                    
                    continue
                
                # compare all possible options
                metric = []
                for subItem1 in trajectory1:
                    
                    for subItem2 in trajectory2:
                        metric.append(DTWDistance_w(subItem1, subItem2, 52))
                
                metric = max(metric)
                
                # store calculated distances
                distanceMatrix_2[index1][index2] = metric
                distanceMatrixDict[unionFilter] = metric 
    
    ######## GROUPING PROCESS ########
    # find the lowest distance and a pair of indexes that provide us this distance
    minValue = np.min(list(distanceMatrixDict.values()))
    
    if minValue > THRESHOLD:
        break
    
    minIndices = np.where(distanceMatrix_2 == minValue)
    minIndices = list(zip(minIndices[0], minIndices[1]))
    
    minIndex = minIndices[0]
    
    # define entities names and values, combine them, 
        # put the combination into the dictionary and remove these single entities from it
    filter1 = list(genre_ts.keys())[minIndex[0]]
    filter2 = list(genre_ts.keys())[minIndex[1]]
    
    trajectory1 = genre_ts.get(filter1)
    trajectory2 = genre_ts.get(filter2)
    
    unionFilter = filter1 + filter2
    sorted(unionFilter)
    
    trajectoryGroup = trajectory1 + trajectory2
    
    genre_ts = {key: value for key, value in genre_ts.items()
                    if all(value not in unionFilter for value in key)}
    
    distanceMatrixDict = {key: value for key, value in distanceMatrixDict.items()
                                if all(value not in unionFilter for value in key)}
    
    genre_ts[unionFilter] = trajectoryGroup
    
    print(iteration, 'finished!')
    iteration += 1
    
    if len(list(genre_ts.keys())) == 1:
        break

1 finished!
2 finished!
3 finished!
4 finished!
5 finished!
6 finished!
7 finished!
8 finished!
9 finished!
10 finished!
11 finished!
12 finished!
13 finished!


#### output matrices

In [287]:
distanceMatrixDict

{('1', '4'): 57.26340137153295,
 ('1', '6'): 112.46283337702923,
 ('1', '7'): 79.00639385287468,
 ('1', '8'): 94.04458186097155,
 ('1', '11'): 40.14863417386116,
 ('1', '15'): 92.04719524352227,
 ('1', '16'): 155.8429332487373,
 ('1', '19'): 48.3214717092815,
 ('1', '20'): 33.73392686672311,
 ('1', '21'): 41.39904180393361,
 ('1', '22'): 69.65840690895968,
 ('1', '25'): 99.9365408619896,
 ('1', '26'): 61.448709536659656,
 ('1', '29'): 48.65870663558559,
 ('1', '30'): 61.52385721391061,
 ('1', '37'): 56.92424103142682,
 ('1', '39'): 96.22116490356977,
 ('1', '40'): 42.9490246193187,
 ('1', '42'): 53.76918128539545,
 ('1', '43'): 94.49616959148904,
 ('1', '44'): 86.38383168492561,
 ('1', '45'): 110.44732601344529,
 ('1', '46'): 133.86788944149478,
 ('4', '6'): 58.360507983508484,
 ('4', '7'): 30.254319607622737,
 ('4', '8'): 42.60622061441744,
 ('4', '11'): 94.83775943306553,
 ('4', '15'): 35.51991122576136,
 ('4', '16'): 93.47629098353335,
 ('4', '19'): 35.43915325289088,
 ('4', '20'): 

In [288]:
distanceMatrix_2

array([[         nan,  57.26340137, 112.46283338, ...,  84.29446712,
         24.23447074,  79.49025125],
       [         nan,          nan,  58.36050798, ..., 139.97806358,
         42.213654  ,  96.30290928],
       [         nan,          nan,          nan, ..., 196.40773916,
         93.50202612, 120.80562901],
       ...,
       [         nan,          nan,          nan, ...,          nan,
        107.19631481, 152.52540772],
       [         nan,          nan,          nan, ...,          nan,
                 nan,  84.79747484],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan]])

#### Clusters

In [289]:
for key, _ in genre_ts.items():
    print(key)

('1',)
('4',)
('6',)
('7',)
('8',)
('11',)
('15',)
('16',)
('19',)
('20',)
('21',)
('22',)
('25',)
('26',)
('29',)
('30',)
('37',)
('39',)
('40',)
('42',)
('43',)
('44',)
('45',)
('46',)
('18', '36')
('2', '3')
('13', '38')
('33', '47')
('24', '48')
('5', '10')
('23', '35')
('31', '32')
('14', '9', '27')
('28', '34')
('0', '17')
('12', '41')


In [283]:
for idx, genre in enumerate(top_genres_str):
    print(idx,": ", genre)

0 :  Rap
1 :  Country
2 :  Pop
3 :  Rock
4 :  RandB
5 :  AlternativeRock
6 :  AdultAlternative
7 :  PopRock
8 :  AdultContemporary
9 :  Soul
10 :  Ballad
11 :  Producer
12 :  HipHop
13 :  Alternative
14 :  Soundtrack
15 :  Electronic
16 :  DancePop
17 :  SoulPop
18 :  Memes
19 :  Dance
20 :  UK
21 :  Canada
22 :  WestCoast
23 :  ElectroPop
24 :  IndieRock
25 :  EastCoast
26 :  Remix
27 :  Screen
28 :  SynthPop
29 :  Piano
30 :  AlternativePop
31 :  PopPunk
32 :  NeoSoul
33 :  Trap
34 :  Dancehall
35 :  PopCountry
36 :  PowerPop
37 :  PostGrunge
38 :  Folk
39 :  ElectronicRock
40 :  SingerSongwriter
41 :  DMV
42 :  IndiePop
43 :  LatinPop
44 :  Reggae
45 :  EnEspa\xc3\xb1ol
46 :  Atlanta
47 :  PunkRock
48 :  DirtySouth
49 :  Acoustic


#### Plot similar genre TS

In [290]:
for key, value in genre_ts.items():
    
    if len(key) == 1:
        continue
    
    figure = make_subplots(rows=1, cols=1)
    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(value))]
    
    for index, subValue in enumerate(value):
        
        figure.add_trace(go.Scatter(x=list(range(0, len(subValue))), y=subValue,
                                    mode='lines', marker_color=colors[index], line = dict(width=4), line_shape='spline'), row=1, col=1,
                        )
         
    figure.update_layout(showlegend=False, height=600, width=900)
    figure.show()